In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os

# === Load dataset ===
file_path = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\AlphabetSignLanguages\FSL\Alphabetical_hand_sign_data.csv"
df = pd.read_csv(file_path)

# === Prepare features and labels ===
X = df.drop("label", axis=1).values
y = pd.Categorical(df["label"]).codes  # convert A–Z labels to numeric codes

# === Split dataset ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === Define Keras model ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(set(y)), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# === Train model ===
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_split=0.2
)

# === Evaluate model ===
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.4f}")

# === Ensure save directory exists ===
SAVE_DIR = "TFModels"
os.makedirs(SAVE_DIR, exist_ok=True)

# === 1️⃣ Save in Keras format (for reloading in Python)
keras_path = os.path.join(SAVE_DIR, "FSL_Alphabet_TF_Model.keras")
model.save(keras_path)
print(f"💾 Saved Keras model at: {keras_path}")

# === 2️⃣ Export as SavedModel (for TFLite conversion or mobile)
export_dir = os.path.join(SAVE_DIR, "FSL_Alphabet_TF_Model_SavedModel")
model.export(export_dir)
print(f"📂 Exported TensorFlow SavedModel at: {export_dir}")

# === 3️⃣ Convert SavedModel → TensorFlow Lite (.tflite)
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_path = os.path.join(SAVE_DIR, "FSL_Alphabet_TF_Model.tflite")
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"📦 TensorFlow Lite model saved at: {tflite_path}")


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from collections import deque
import os

# === Model paths ===
KERAS_PATH = "TFModels/FSL_Alphabet_TF_Model.keras"
SAVEDMODEL_PATH = "TFModels/FSL_Alphabet_TF_Model_SavedModel"
TFLITE_PATH = "TFModels/FSL_Alphabet_TF_Model.tflite"

# === Auto-detect model type ===
USE_TFLITE = False
USE_SAVEDMODEL = False

if os.path.exists(KERAS_PATH):
    try:
        model = tf.keras.models.load_model(KERAS_PATH)
        print("✅ Loaded Keras model.")
    except Exception as e:
        print(f"⚠️ Failed to load .keras model: {e}")
        USE_SAVEDMODEL = True
elif os.path.exists(SAVEDMODEL_PATH):
    try:
        model = tf.keras.models.load_model(SAVEDMODEL_PATH)
        print("✅ Loaded SavedModel.")
        USE_SAVEDMODEL = True
    except Exception as e:
        print(f"⚠️ Failed to load SavedModel: {e}")
        USE_TFLITE = True
else:
    print("⚠️ Loading TensorFlow Lite model...")
    USE_TFLITE = True

# === Setup TensorFlow Lite interpreter if needed ===
if USE_TFLITE:
    interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print("✅ Loaded TensorFlow Lite model.")

# === Mediapipe setup ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# === Labels (A–Z) ===
labels = [chr(i) for i in range(65, 91)]

# === Webcam setup ===
cap = cv2.VideoCapture(0)
history = deque(maxlen=10)

# Keep track of last valid prediction
last_prediction = "No hands detected"
no_hand_counter = 0
NO_HAND_THRESHOLD = 15   # number of frames before showing "No hands detected"

print("🎥 Starting real-time prediction... (Press ESC to exit)")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    current_prediction = None

    if results.multi_hand_landmarks:
        no_hand_counter = 0  # reset if hand is detected
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten().reshape(1, -1)

            try:
                if USE_TFLITE:
                    interpreter.set_tensor(input_details[0]['index'], landmarks.astype(np.float32))
                    interpreter.invoke()
                    output = interpreter.get_tensor(output_details[0]['index'])
                else:
                    output = model.predict(landmarks, verbose=0)

                pred_idx = np.argmax(output)
                current_prediction = labels[pred_idx] if pred_idx < len(labels) else "?"
            except Exception as e:
                current_prediction = f"Error: {e}"

            history.append(current_prediction)
    else:
        # increment "no hand" counter
        no_hand_counter += 1

    # smooth prediction
    if history:
        smoothed_prediction = max(set(history), key=history.count)
        if current_prediction:
            last_prediction = smoothed_prediction
        elif no_hand_counter > NO_HAND_THRESHOLD:
            last_prediction = "No hands detected"

    # display result
    cv2.putText(frame, f"Predicted: {last_prediction}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("ASL Alphabet Recognition (TensorFlow)", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
print("🛑 Stopped real-time prediction.")
